# Mẹo với SQL

## Kiểm tra kiểu dữ liệu trong bảng

```sql
SELECT COLUMN_NAME, DATA_TYPE 
FROM INFORMATION_SCHEMA.COLUMNS 
WHERE TABLE_NAME = 'ACCOUNT' -- Điền tên bảng

```


## Dữ liệu bị collate

- Khi làm việc, dữ liệu các bảng có thể bị convert định dạng khác nhau. Trong trường hợp này, phải dùng `COLLATE DATABASE_DEFAULT`


```sql
select top 10 * from 
INFORMATION_SCHEMA.columns
where table_name COLLATE DATABASE_DEFAULT in (select new_table COLLATE DATABASE_DEFAULT 
from table_rename
)

```

## Drop nhiều bảng theo điều kiện



```sql
declare @string nvarchar(max) 
SET @STRING = ''
SELECT @STRING = @STRING + 'DROP TABLE ANALYTICS.DBO.' + TABLE_NAME + ';' 
FROM ANALYTICS.INFORMATION_SCHEMA.TABLES 
WHERE TABLE_NAME like 'Mckinsey%'
exec (@string)
```

## Auto Wrap in MSSQL

Tools >> Options >> Text Editor >> Transact SQL >> General >> WordWrap

## Format định dạng thêm ký tự 00 trước số


```sql
select FORMAT(01,'00#') as index


```

## Kiểm tra định dạng dữ liệu một bảng



```sql
-- Với Link server
select * from ANHNT67.DATA.INFORMATION_SCHEMA.COLUMNS where TABLE_NAME='CARD_LIVE'

-- Với Local
select * from COLUMNS where TABLE_NAME='CARD_LIVE'

-- Kiểm tra toàn bộ thông tin
sp_help table_name 
sp_help CARD_LIVE

```

## Convert từ tiếng Việt có dấu sang không dấu



```sql
CREATE FUNCTION [dbo].[fChuyenCoDauThanhKhongDau](@inputVar NVARCHAR(MAX) )
RETURNS NVARCHAR(MAX)
AS
BEGIN    
    IF (@inputVar IS NULL OR @inputVar = '')  RETURN ''
   
    DECLARE @RT NVARCHAR(MAX)
    DECLARE @SIGN_CHARS NCHAR(256)
    DECLARE @UNSIGN_CHARS NCHAR (256)

    SET @SIGN_CHARS = N'ăâđêôơưàảãạáằẳẵặắầẩẫậấèẻẽẹéềểễệếìỉĩịíòỏõọóồổỗộốờởỡợớùủũụúừửữựứỳỷỹỵýĂÂĐÊÔƠƯÀẢÃẠÁẰẲẴẶẮẦẨẪẬẤÈẺẼẸÉỀỂỄỆẾÌỈĨỊÍÒỎÕỌÓỒỔỖỘỐỜỞỠỢỚÙỦŨỤÚỪỬỮỰỨỲỶỸỴÝ' + NCHAR(272) + NCHAR(208)
    SET @UNSIGN_CHARS = N'aadeoouaaaaaaaaaaaaaaaeeeeeeeeeeiiiiiooooooooooooooouuuuuuuuuuyyyyyAADEOOUAAAAAAAAAAAAAAAEEEEEEEEEEIIIIIOOOOOOOOOOOOOOOUUUUUUUUUUYYYYYDD'

    DECLARE @COUNTER int
    DECLARE @COUNTER1 int
   
    SET @COUNTER = 1
    WHILE (@COUNTER <= LEN(@inputVar))
    BEGIN  
        SET @COUNTER1 = 1
        WHILE (@COUNTER1 <= LEN(@SIGN_CHARS) + 1)
        BEGIN
            IF UNICODE(SUBSTRING(@SIGN_CHARS, @COUNTER1,1)) = UNICODE(SUBSTRING(@inputVar,@COUNTER ,1))
            BEGIN          
                IF @COUNTER = 1
                    SET @inputVar = SUBSTRING(@UNSIGN_CHARS, @COUNTER1,1) + SUBSTRING(@inputVar, @COUNTER+1,LEN(@inputVar)-1)      
                ELSE
                    SET @inputVar = SUBSTRING(@inputVar, 1, @COUNTER-1) +SUBSTRING(@UNSIGN_CHARS, @COUNTER1,1) + SUBSTRING(@inputVar, @COUNTER+1,LEN(@inputVar)- @COUNTER)
                BREAK
            END
            SET @COUNTER1 = @COUNTER1 +1
        END
        SET @COUNTER = @COUNTER +1
    END
    -- SET @inputVar = replace(@inputVar,' ','-')
    RETURN @inputVar
END

```

- Ví dụ



```sql
select dbo.fChuyenCoDauThanhKhongDau(N'Một thiên thần') 


```

## Functions chọn giá trị lớn hơn



```sql
/********************************
Script:         Function lấy giá trị lớn hơn
Author:         TrangTQ2
Created date:   20190301
Modifed date:   
========================================
Description: Tạo fuction lấy giá trị lớn hơn trong 2 số

*********************************/

create function [dbo].[GREATER](@val1 FLOAT, @val2 FLOAT)
returns FLOAT
as
begin
  if @val1 > @val2
    return @val1
  return isnull(@val2,@val1)
end
GO

```

## Procedure pivot vs. unpivot



```sql
/********************************
Script:         Procedure pivot và unpivot
Author:         TrangTQ2
Created date:   20190301
Modifed date:   
========================================
Description: Tạo procedure pivot và unpivot trong SQL với concept giống dcast và melt trong R

*********************************/

---------------------------------------------------------CREATE DATA TO TEST---------------------------------------------------

create table ##student_wide
(
id int,
mark1 float,
mark2 float,
mark3 float
)

insert into ##student_wide values (1, 1,2,3)
insert into ##student_wide values (2, 5,6,7)
insert into ##student_wide values (3, 7,4,2)

select * from ##student_wide

SELECT * 
into ##student_long FROM
(SELECT *
FROM ##student_wide) a
UNPIVOT
(value_var FOR group_var IN ([mark1], [mark2], [mark3])
) b

select * from ##student_long

select * from 
(select * from ##student_long) a
pivot 
(sum(value_var) for group_var in ([mark1], [mark2], [mark3])) b

---------------------------------------------------CREATE PROCEDURE TO PIVOT---------------------------------------------

--drop procedure if exists sql_pivot
create procedure sql_pivot  @data nvarchar(max),
                            @group nvarchar(max), -- Variable to pivot
                            @row nvarchar(max), -- Column to keep as key
                            @value nvarchar(max), -- Column to calculate
                            @function nvarchar(max) = 'sum', -- Function to calculate
                            @stored_data nvarchar(max) = '##pivot_result' -- Store data result
as
begin
declare @sql nvarchar(max)
set @sql = '
declare @group_value nvarchar(max)
set @group_value = STUFF(
(SELECT '','' + ''['' + b.'+@group+' + '']''
              FROM 
              (select distinct '+ @group +' from '+ @data +') b
              FOR XML PATH ('''')),1,1,'''')

declare @sqlstr nvarchar(max)
set @sqlstr = N''
drop table if exists '+ @stored_data +'
select * 
into '+ @stored_data +'
from 
(select '+ @group+ ', ' + @row + ', '+@value +'  from '+ @data +') a
pivot 
('+@function+'('+ @value +') for '+@group+' in ('' + @group_value + '')) b
''
exec sp_executesql @sqlstr
'
exec sp_executesql @sql
end;


sql_pivot @data = '##student_long', 
          @row = 'id',
          @group= 'group_var', 
          @value = 'value_var', 
          @function = 'sum',
          @stored_data = '##student_wide2'

select * from ##student_long
select * from ##pivot_result
select * from ##student_wide2

---------------------------------------------------CREATE PROCEDURE TO UNPIVOT---------------------------------------------

--drop procedure if exists sql_unpivot
create procedure sql_unpivot 
 @db nvarchar(max) = 'tempdb', --Database contains data
@data nvarchar(max), -- Name of data
@id_var nvarchar(max), -- Key variables
@group_var nvarchar(max) = 'group_var', --Name of groups variable
@value_var nvarchar(max) = 'value_var', -- Name of value variable
@stored_data nvarchar(max) = '##unpivot_result'  -- Data storage
as 
begin
declare @sql nvarchar(max) 
set @sql = N'
drop table if exists ##all_cols
select name into ##all_cols
from '+@db+'.sys.columns 
where object_id in (select object_id from '+@db+'.sys.tables where name like ''%'+@data+''')

declare @cols AS NVARCHAR(MAX)
   select @cols = STUFF((SELECT '','' + QUOTENAME(name) 
                    FROM (
                        select name from ##all_cols 
                        where name not in ('+@id_var+') 
                        ) cols            
                    FOR XML PATH(''''), TYPE).value(''.'', ''NVARCHAR(MAX)''),1,1,'''')
-- Show columns
print @cols 

declare @key AS NVARCHAR(MAX)
   select @key = STUFF((SELECT '','' + QUOTENAME(name) 
                    FROM (
                        select name from ##all_cols 
                        where name in ('+@id_var+') 
                        ) cols            
                    FOR XML PATH(''''), TYPE).value(''.'', ''NVARCHAR(MAX)''),1,1,'''')
-- Show key
print @key 

--Run procedure

declare @sqlstr nvarchar(max)
set @sqlstr = N''
            drop table if exists '+@stored_data+'
            select ''+@key+'', '+@group_var+', '+@value_var+'
            into '+@stored_data+'
            from(
               select *
               from '+@db+'.dbo.['+@data+']
            ) as a
            unpivot
            (
              '+@value_var+' for '+@group_var+' in ('' + @cols + '')
            ) as b
            select * from '+@stored_data+'
            ''
print @sqlstr
exec sp_executesql @sqlstr
'
exec sp_executesql @sql
end

sql_unpivot @db = 'tempdb', 
          @data = '##student_wide', 
          @id_var = '''id''',  --> neu muon giu nhieu cot thi viet '''id'',''mark1'''
          @stored_data = '##unpivot_result', 
          @group_var = 'group_mark'

select * from ##student_wide
select * from ##unpivot_result
select * from ##student_long

```

## Đổi tên bảng và tên cột



```sql
/********************************
Script:         Đổi tên bảng và cột
Author:         TrangTQ2
Created date:   20190301
Modifed date:   
========================================
Description:

*********************************/

drop table ##test
drop table draft.dbo.test
drop table draft.dbo.test2

select top 10 * into ##test from m74.bicdata.dbo.customer
select top 10 * into draft.dbo.test from m74.bicdata.dbo.customer

select * from ##test
select * from draft.dbo.test2

-- Đổi tên bảng (không đổi được tên bảng tạm)

use DBS_REGULAR
go
sp_rename 'PCB_UPLTOPUP_FULL_201806','PCB_UPLTOPUP_FULL'
go

-- Đổi tên cột 
use DBS_ADHOC
go
sp_rename 'EBANKING_TRANSACTION_2017.TRANS_TYPE','TYPE','column'
--tempdb.sys.sp_rename '##test.RECID','CIF','column'
go

```

## Viết hoa chữ đầu



```sql
/********************************
Script:         Function viết hoa chữ đầu
Author:         TrangTQ2
Created date:   20190301
Modifed date:   
========================================
Description: Tạo function viết hoa chữ cái đầu

*********************************/

CREATE FUNCTION dbo.fn_capitalize
(
@str AS nvarchar(100)
)
RETURNS nvarchar(100)
AS
BEGIN

DECLARE
@ret_str AS varchar(100),
@pos AS int,
@len AS int

SELECT
@ret_str = N' ' + LOWER(@str),
@pos = 1,
@len = LEN(@str) + 1

WHILE @pos > 0 AND @pos < @len
BEGIN
SET @ret_str = STUFF(@ret_str,
@pos + 1,
1,
UPPER(SUBSTRING(@ret_str,@pos + 1, 1)))
SET @pos = CHARINDEX(N' ', @ret_str, @pos + 1)
END

RETURN RIGHT(@ret_str, @len - 1)

END

------
SELECT dbo.fn_capitalize('Average Spending per Transaction')

```

## Tạo floor date



```sql
/********************************
Script:         Floor Date trong SQL
Author:         AnhHD3
Created date:   20190412
Modifed date:   
========================================
Description:  Hàm cho phép lấy nhanh ngày đầu tiên 
của tháng, năm, cho phép tính toán và nhóm theo month/year
*********************************/

IF OBJECT_ID('fn_FloorDate') IS NOT NULL DROP FUNCTION fn_FloorDate
SET ANSI_NULLS OFF
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE FUNCTION [dbo].[fn_FloorDate] (
  @Date DATETIME = NULL,
  @DatePart VARCHAR(6) = 'day'
)
RETURNS DATETIME
AS
BEGIN
  IF (@Date IS NULL)
    SET @Date = GETDATE();

  RETURN
  CASE
    WHEN LOWER(@DatePart) = 'year' THEN DATEADD(YEAR, DATEDIFF(YEAR, 0, @Date), 0)
    WHEN LOWER(@DatePart) = 'month' THEN DATEADD(MONTH, DATEDIFF(MONTH, 0, @Date), 0)
    WHEN LOWER(@DatePart) = 'day' THEN DATEADD(DAY, DATEDIFF(DAY, 0, @Date), 0)
    WHEN LOWER(@DatePart) = 'hour' THEN DATEADD(HOUR, DATEDIFF(HOUR, 0, @Date), 0)
    WHEN LOWER(@DatePart) = 'minute' THEN DATEADD(MINUTE, DATEDIFF(MINUTE, 0, @Date), 0)
    WHEN LOWER(@DatePart) = 'second' THEN DATEADD(SECOND, DATEDIFF(SECOND, '2000-01-01', @Date), '2000-01-01')
    ELSE DATEADD(DAY, DATEDIFF(DAY, 0, @Date), 0)
  END;
END

-- Example

select getdate() , dbo.fn_FloorDate(getdate(), 'month')

```

## Clean city



```sql
/********************************
Script:         Function clean city
Author:         Hoangnb3
Created date:   20190426
Modifed date:   
========================================
Description: Tạo function làm sạch dữ liệu tỉnh/thành phố trực thuộc trung ương

Note: Sử dụng sau khi đã dùng function chuyển sang không dấu

*********************************/

USE [master]
GO

SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO

ALTER FUNCTION [dbo].[fn_cleancity](@inputVar NVARCHAR(MAX) )
returns nvarchar(max)
as
begin
    declare @city_new nvarchar(max)

    set @city_new = CASE WHEN @inputVar like '%ha%no%' or @inputVar like '%ha%n%i%' or @inputVar like '%ha dong%'
                            or @inputVar like '%hn%' or @inputVar like '%hn' or @inputVar = 'hn'
                            or @inputVar like '%h anoi%' or @inputVar like '%hanoi%' or @inputVar like '%ha n oi%'
                            or @inputVar like '%ha ná»i%' or @inputVar like '%ha nnoi%' or @inputVar like '%ha no%'
                            or @inputVar like '%ha npoi%' or @inputVar like '%ha tay%' or @inputVar like '%hã  ná»?i%'
                            or @inputVar like '%hã¡ noi%' or @inputVar like '%ba vi%' or @inputVar like '%gia lam%'
                            or @inputVar like '%ha hoi%' or @inputVar like '%hoan kiem%' or @inputVar like '%Van Chuong%'
                            or @inputVar like '%Trung Kinh%' or @inputVar like '%Trung Hoa%' or @inputVar like '%Quoc Oai%'
                            or @inputVar like '%Van Dien%' or @inputVar like '%Thach That%' or @inputVar like '%Cau Giay%'
                            or @inputVar like '%Hoang Mai%' or @inputVar like '%Phu Do%' or @inputVar like '%Me Linh%'
                            or @inputVar like '%Ba Dinh%'
                            or @inputVar like '%Quoc Oai%' THEN 'ha noi'
                        WHEN @inputVar like '%chi%minh%' or @inputVar like '%hcm%' or @inputVar like '%quan 1%'
                            or @inputVar like '%há»? chã­ minh%' or @inputVar like '%tp h cm%' or @inputVar like '%tan phu%'
                            or @inputVar like '%thu duc%' or @inputVar like '%go vap%' or @inputVar like '%Tan Binh%'
                            or @inputVar like '%Chi Hoa%' or @inputVar like '%Hoc Mon%' or @inputVar like '%Tan Thuan Dong%'
                            or @inputVar like '%Binh Hung Hoa%' or @inputVar like '%Ap Sai Gon%' or @inputVar like '%Cu Chi%'
                            or @inputVar like '%Phu Nhuan%' or @inputVar like '%Binh Tan%' or @inputVar like '%Nha Be%'
                            or @inputVar like '%Linh Xuan%'
                            or @inputVar like '%Trung My Tay%' THEN 'ho chi minh'
                        WHEN @inputVar like '%quang%ninh%' or @inputVar like '%mong cai%' or @inputVar like '%quag ninh%'
                            or @inputVar like '%ha long%' or @inputVar like '%dong trieu%' or @inputVar like '%Van Don%'
                            or @inputVar like '%Quang Yen%' or @inputVar like '%Q%Ninh%' or @inputVar like '%Tien Yen%'
                            or @inputVar like '%Hoanh Bo%'
                            or @inputVar like '%cam pha%' THEN 'quang ninh'
                        WHEN @inputVar like '%quang binh%' or @inputVar like '%quangb binh%' or @inputVar like '%q%binh%' 
                            or @inputVar like '%Hoan Lao%' or @inputVar like '%Dong Hoi%'
                            or @inputVar like '%qbinh%' THEN 'quang binh'
                        WHEN @inputVar like '%quang tri%' or @inputVar like '%quangtri%' or @inputVar like '%q%tr%'
                            or @inputVar like '%quáº£ng tri%' or @inputVar like '%Trieu Phong%' or @inputVar like '%Ho Xa%'
                            or @inputVar like '%Dong Ha%' THEN 'quang tri'
                        WHEN @inputVar like '%hue%' or @inputVar like '%Phu Loc%'
                            or @inputVar like '%th%hu%' THEN 'hue'
                        WHEN @inputVar like '%tien giang%' or @inputVar like '%t%giang%' or @inputVar like '%My Tho%'
                            or @inputVar like '%Cai Be%' or @inputVar like '%Go Cong%'
                            or @inputVar like '%tiá»n giang%' THEN 'tien giang'
                        WHEN @inputVar like '%b%tre%' or @inputVar like '%Mo Cay%' or @inputVar like '%Giong Trom%'
                            or @inputVar like '%ben tre%' THEN 'ben tre'
                        WHEN @inputVar like '%ca mau%' THEN 'ca mau'
                        WHEN @inputVar like '%binh phuoc%' or @inputVar like '%Dong Xoai%' or @inputVar like '%Binh Long%'
                            or @inputVar like '%Loc Ninh%'
                            or @inputVar like '%binh%ph%' THEN 'binh phuoc'
                        WHEN @inputVar like '%binh dinh%' or @inputVar like '%binh%d%' or @inputVar like '%An Nhon%'
                            or @inputVar like '%Bong Son%'
                            or @inputVar like '%Qu%Nhon%' THEN 'binh dinh'
                        WHEN @inputVar like '%phu tho%' or @inputVar like '%phutho%' or @inputVar like '%phu%th%'
                            or @inputVar like '%Cam Khe%'
                            or @inputVar like '%viet tri%' THEN 'phu tho'
                        WHEN @inputVar like '%an%giang%' or @inputVar like '%An Phu%' or @inputVar like '%Tan Chau%'
                            or @inputVar like '%Long Xuyen%' THEN 'an giang'
                        WHEN @inputVar like '%yen bai%'
                            or @inputVar like '%Yen Binh%' THEN 'yen bai'
                        WHEN @inputVar like '%son la%'
                            or @inputVar like '%Moc Chau%' THEN 'son la'
                        WHEN @inputVar like '%tay ninh%' or @inputVar like '%Ben Cau%'
                            or @inputVar like '%Trang Bang%' THEN 'tay ninh'
                        WHEN @inputVar like '%vung%tau%' or @inputVar = 'ba ria- vå©ng tã u' or @inputVar like '%Ba Ria%'
                            or @inputVar like '%Xuyen Moc%' THEN 'vung tau'
                        WHEN @inputVar like '%bac can%' or @inputVar like '%b%c%c%n%' or @inputVar like '%b%c%k%n%'
                            or @inputVar like '%bac kan%' THEN 'bac can'
                        WHEN @inputVar like '%b%lieu%'
                            or @inputVar like '%Gia Rai%' THEN 'bac lieu'
                        WHEN @inputVar like '%bac ninh%' or @inputVar like '%b%ninh%' or @inputVar like '%yen trung%'
                            or @inputVar like '%Tu Son%' or @inputVar like '%Gia Binh%'
                            or @inputVar like '%Thuan Thanh%' THEN 'bac ninh'
                        WHEN @inputVar like '%binh duong%' or @inputVar like '%Thuan An%' or @inputVar like '%Tan Uyen%'
                            or @inputVar like '%Ben Cat%'
                            or @inputVar like '%Thu Dau Mot%' THEN 'binh duong'
                        WHEN @inputVar like '%ha nam%' or @inputVar like '%Ly Nhan%' or @inputVar like '%Kim Bang%'
                            or @inputVar like '%Phu%L%' THEN 'ha nam'
                        WHEN @inputVar like '%binh thuan%' or @inputVar like '%phan thiet%' or @inputVar like '%binh%th%'
                            or @inputVar like '%Duc Linh%'
                            or @inputVar = 'phãº thá»' THEN 'binh thuan'
                        WHEN @inputVar like '%can tho%' or @inputVar like '%c%tho%' or @inputVar like '%Phong Dien%'
                            or @inputVar like '%Xuan Khanh%' THEN 'can tho'
                        WHEN @inputVar like '%hau giang%' or @inputVar like '%h%giang%' or @inputVar like '%Phung Hiep%'
                            or @inputVar like '%Rach Goi%'
                            or @inputVar like '%Vi Thanh%' THEN 'hau giang'
                        WHEN @inputVar like '%tuyen quang%' or @inputVar like '%Chiem Hoa%'
                            or @inputVar like '%tuyãªn quang%' THEN 'tuyen quang'
                        WHEN @inputVar like '%thai nguyen%' or @inputVar like '%Phu Dinh%'
                            or @inputVar like '%thainguyen%' THEN 'thai nguyen'
                        WHEN @inputVar like '%thai binh%' or @inputVar like '%Hung Ha%' or @inputVar like '%Tien Hai%'
                            or @inputVar like '%thai bã¬nh%' THEN 'thai binh'
                        WHEN @inputVar like '%hai phong%' or @inputVar like '%h%ph%' or @inputVar like '%An Bien%'
                            or @inputVar like '%Kien An%'
                            or @inputVar like '%hp%' THEN 'hai phong'
                        WHEN @inputVar like '%da nang%' or @inputVar like '%da%n%' or @inputVar = 'da nã¢ng' 
                            or @inputVar like '%d%nang%' or @inputVar like '%Lien Chieu%' or @inputVar like '%Hai Chau%'
                            or @inputVar like '%Son Tra%' or @inputVar like '%Cam Le%' or @inputVar like '%Hoa Vang%'
                            or @inputVar = 'cam ranh' THEN 'da nang'
                        WHEN @inputVar like '%dac lac%' or @inputVar like '%d%c%l%c%' or @inputVar like '%dak dak%'
                            or @inputVar like '%dak lak%' or @inputVar like '%dakalk%' or @inputVar like '%dakalk%'
                            or @inputVar like '%daklak%' or @inputVar like '%dallak%' or @inputVar like '%buon ma thuot%'
                            or @inputVar like '%Ban Me Thuot%'
                            or @inputVar like '%Buon Don%' THEN 'dac lac'
                        WHEN @inputVar like '%dac nong%' or @inputVar like '%d%nong%' or @inputVar like '%Gia Nghia%'
                            OR @inputVar like '%dak nong%' THEN 'dac nong'
                        WHEN @inputVar like '%bac giang%' or @inputVar like '%b%giang%' or @inputVar like '%yen the%'
                            or @inputVar like '%Hiep Hoa%'
                            or @inputVar like '%Viet Yen%' THEN 'bac giang'
                        WHEN @inputVar like '%bac ninh%' or @inputVar like '%Yen Phong%' or @inputVar like '%Luong Tai%'
                            or @inputVar like '%ban ninh%' THEN 'bac ninh'
                        WHEN @inputVar like '%dong nai%' or @inputVar like '%d%nai%' or @inputVar like '%Long Thanh%'
                            or @inputVar like '%Long Khanh%' or @inputVar like '%Ho Nai%'
                            or @inputVar like '%Bien Hoa%' THEN 'dong nai'
                        WHEN @inputVar like '%vinh long%' or @inputVar like '%Tra On%'
                            OR @inputVar like '%vfinh long%' THEN 'vinh long'
                        WHEN @inputVar like '%lai chau%' THEN 'lai chau'
                        WHEN @inputVar like '%lam dong%' or @inputVar like '%lam%d%' or @inputVar like '%Bao Loc%'
                            or @inputVar like '%Da Lat%' THEN 'lam dong'
                        WHEN @inputVar like '%lang son%' or @inputVar like '%l%son%' or @inputVar like '%Van Linh%'
                            or @inputVar like '%Xuan Tinh%' THEN 'lang son'
                        WHEN @inputVar like '%lao cai%' THEN 'lao cai'
                        WHEN @inputVar like '%dong nai%'
                            or @inputVar like '%Xuan Loc%' THEN 'dong nai'
                        WHEN @inputVar like '%tay ninh%'
                            or @inputVar like '%Phuoc Dong%' THEN 'tay ninh'
                        WHEN @inputVar like '%ninh binh%' THEN 'ninh binh'
                        WHEN @inputVar like '%ha giang%'
                            or @inputVar like '%ha ginag%' THEN 'ha giang'
                        WHEN @inputVar like '%thanh hoa%' or @inputVar like '%thanh oai%' or @inputVar like '%thanh ho%'
                            or @inputVar like '%Yen Dinh%' or @inputVar like '%Bim Son%' or @inputVar like '%Nong Cong%'
                            or @inputVar like '%Tho Xuan%'
                            or @inputVar like '%Quang Xuong%' THEN 'thanh hoa'
                        WHEN @inputVar like '%tra%vinh%' or @inputVar like '%Tieu Can%'
                            or @inputVar like '%Duyen Hai%' THEN 'tra vinh'
                        WHEN @inputVar like '%ha tinh%' or @inputVar like '%ha ti nh%' or @inputVar like '%ha tin%'
                            or @inputVar like '%hã¡ tinh%'
                            or @inputVar like '%hat inh%' THEN 'ha tinh'
                        WHEN @inputVar like '%nghe an%' or @inputVar like '%nghá»? an%' or @inputVar like '%vinh%'
                            or @inputVar like '%Ngh%an%' or @inputVar like '%Dien Chau%' or @inputVar like '%Quynh Luu%'
                            or @inputVar like '%Do Luong%' or @inputVar like '%Minh Son%'
                            or @inputVar like '%Yen Thanh%' THEN 'nghe an'
                        WHEN @inputVar like '%nam dinh%' or @inputVar like '%nam dinnh%' or @inputVar like '%nam%d%'
                            or @inputVar like '%Vu Ban%'
                            or @inputVar like '%yen thang%' THEN 'nam dinh'
                        WHEN @inputVar like '%hai duong%' or @inputVar like '%Kinh Mon%' or @inputVar like '%Cam Giang%'
                            or @inputVar like '%h%duong%' THEN 'hai duong'
                        WHEN @inputVar like '%hoa binh%'
                            or @inputVar like '%hoabinh%' THEN 'hoa binh'
                        WHEN @inputVar like '%hung yen%' or @inputVar like '%Yen My%' or @inputVar like '%Khoai Chau%'
                            or @inputVar like '%Tien Lu%'
                            or @inputVar like '%Van Lam%' THEN 'hung yen'
                        WHEN @inputVar like '%kien giang%' or @inputVar like '%Rach Gia%' or @inputVar like '%Duong Dong%'
                            or @inputVar like '%phu quoc%'
                            or @inputVar like '%kie giang%' THEN 'kien giang'
                        WHEN @inputVar like '%kon tum%'
                            or @inputVar like '%kom tum%' THEN 'kon tum'
                        WHEN @inputVar like '%khanh hoa%' or @inputVar like '%khanh hioa%' or @inputVar like '%nha trang%'
                            or @inputVar like '%Cam Lam%' or @inputVar like '%Ninh Hoa%'
                            or @inputVar like '%Dien Khanh%' THEN 'khanh hoa'
                        WHEN @inputVar like '%vinh phuc%' OR @inputVar like '%phuc yen%'
                            OR @inputVar like '%Yen Lac%' THEN 'vinh phuc'
                        WHEN @inputVar like '%long an%' or @inputVar like '%lon gan%' or @inputVar like '%Tan Tru%'
                            or @inputVar like '%Tan An%' or @inputVar like '%Ben Luc%' or @inputVar like '%Can Giuoc%'
                            or @inputVar like '%Can Duoc%' or @inputVar like '%An Thanh%' or @inputVar like '%Tan Hung%'
                            or @inputVar like '%Moc Hoa%'
                            or @inputVar like '%lon gan%' THEN 'long an'
                        WHEN @inputVar like '%dong nai%'
                            or @inputVar like '%Dinh Quan%' THEN 'dong nai'
                        WHEN @inputVar like '%ninh binh%'
                            or @inputVar like '%ninhbinh%' THEN 'ninh binh'
                        WHEN @inputVar like '%ninh%th%'
                            or @inputVar like '%Phan Rang%' THEN 'ninh thuan'
                        WHEN @inputVar like '%soc trang%' THEN 'soc trang'
                        WHEN @inputVar like '%phu yen%' or @inputVar like '%Tay Hoa%'
                            or @inputVar like '%Tuy Hoa%' THEN 'phu yen'
                        WHEN @inputVar like '%cao%b%' THEN 'cao bang'
                        WHEN @inputVar like '%d%bien%' THEN 'dien bien'
                        WHEN @inputVar like '%dong thap%' or @inputVar like '%Cao Lanh%' or @inputVar like '%Hong Ngu%'
                            or @inputVar like '%Sa Dec%'
                            or @inputVar like '%d%thap%' THEN 'dong thap'
                        WHEN @inputVar like '%gia lai%' or @inputVar like '%Phu Tuc%' or @inputVar like '%Chu Se%'
                            or @inputVar like '%Pleiku%' THEN 'gia lai'
                        WHEN @inputVar like '%Q%Nam%' or @inputVar like '%Tam Ky%' or @inputVar like '%Duy Xuyen%'
                            or @inputVar like '%Hoi An%' THEN 'quang nam'
                        WHEN @inputVar like '%Q%Ngai%' THEN 'quang ngai'
                        ELSE 'Others'
                        END
    
    return @city_new
end

```

## Kiểm tra chất lượng dữ liệu hàng ngày



```sql
/********************************
Script: Tạo bảng kiểm tra chất lượng dữ liệu (nrow) hàng ngày
Author:         AnhHD3
Created date:   20180701
========================================
Step 1: Sửa tên Database cho phù hợp
Step 2: Run hoặc tạo job hàng ngày
*/

-- Step 1: Tạo bảng TABLE_QUALITY

create table DATA.dbo.TABLE_QUALITY
(
TABLE_NAME nvarchar(100), 
N_ROW int)

-- Step 2: Update số dòng trong bảng (chỉ dùng bảng mới)

declare @sql nvarchar(max)
set @sql = ''
select @sql = @sql + '
INSERT INTO DATA.dbo.TABLE_QUALITY (TABLE_NAME, N_ROW)
select ''' + TABLE_NAME + ''', count(1) as N_ROW from DATA.dbo.' + TABLE_NAME + '
'
from DATA.INFORMATION_SCHEMA.tables
where table_name not in (select table_name from data.dbo.table_quality)
print @sql
exec(@sql)

select * from DATA..TABLE_QUALITY

```

## Sửa default template trong SQL

- Sửa file `SQLfile.sql` tại đường dẫn sau `C:\Program Files (x86)\Microsoft SQL Server\130\Tools\Binn\ManagementStudio\SqlWorkbenchProjectItems\Sql`

- Nội dung default script


```sql
/********************************
Script:			Giao dịch Ebanking
Author:			AnhHD3
Created date:	20180701
Modifed date:	20181127
========================================
Description: Create Data Preparation 
for datamart
========================================
Affected tables:
	- [dbo].[POS_ECOMCARD_2017]

Steps:
	- Step 1: Declare date
	- Step 2: 
		- POS transaction
		- ATM/CDM transaction
*********************************/
SELECT TOP 1 * FROM XXX

```

## Bắt đầu têm bảng với DIM

- Tên các bảng thường bắt đầu bằng Dim - viết tắt của Dimension

## Xóa bảng nếu tồn tại


```sql
drop table if exists CROSS_SELLING_RESULTS


```

## Tìm kiếm kiểu dữ liệu trong bảng của db



```sql
-- Kiểm tra thông tin kiểu dữ liệu trong bảng
select table_name, column_name, data_type from [learningsql].[INFORMATION_SCHEMA].[columns]

```

## Sử dụng drop và create trong cùng batch

- DROP và CREATE không thể chạy cùng trong 1 batch, khi muốn sử dụng phải thêm câu lệnh GO


```sql
drop table if exists #temp
GO

create table #temp(abc int)
insert into #temp(abc)
values (1), (2)
GO

```

## Phân biệt định dạng integer

- SQL không phân biệt được định dạng numeric & integer ngay lập tức


```sql
select 7/10  -- Trả kết quả 0

--So sánh

select 7.0/10 -- Chạy đúng

```

## VARCHAR vs. NVARCHAR

- varchar vs. nvarchar: nvarchar cho phép lưu trữ ký tự Unicode, varchar không cho phép
- varchar(n) vs. varchar(max): 


+--------------------+----------------------------+
|     varchar(n)     |        varchar(max)        |
+====================+============================+
| Tối đa 8000 ký tự  | Tối đa hơn 2 tỷ, khoảng 2G |
+--------------------+----------------------------+
| Cho phép tạo index | Không cho phép tạo index   |
+--------------------+----------------------------+


```sql
--Varchar(50): Work
CREATE TABLE dbo.Employee
(id INT identity(1,1)
   PRIMARY KEY,
 Name VARCHAR(50))
GO
CREATE INDEX IX_EmployeeName 
 ON dbo.Employee(Name)
GO

--Varchar(max): Not work
CREATE TABLE dbo.Employee
(id INT identity(1,1)
   PRIMARY KEY,
 Name VARCHAR(Max))
GO
CREATE INDEX IX_EmployeeName
 ON dbo.Employee(Name)
GO 

```

## Primary key vs. Unique key


+----------+----------------------------+-------------------------------------+
| Tiêu chí |        Primary key         |             Unique key              |
+==========+============================+=====================================+
| NULL     |  Không cho phép null       | Cho phép MỘT giá trị NULL           |
+----------+----------------------------+-------------------------------------+
| Index    | Default là clustered index | Default là unique non-cluster index |
+----------+----------------------------+-------------------------------------+
| Limit    | Chỉ có 1 key               | Có thể có nhiều key                 |
+----------+----------------------------+-------------------------------------+


## EXEC vs. EXEC()

- Exec dùng để thực hiện các procedure. VD: `exec sp_pivot`
- Exec() dùng để thực hiện dinamic SQL. 
    


```sql
exec('Select top 10 * from @table')


```

## Xem tất cả các user-defined procedure



```sql
SELECT name, 
       type
  FROM dbo.sysobjects
 WHERE (type = 'P') and
 name like 'u%'

```

    
## Làm việc với link server

**Vấn đề**: Cách link server thủ công (GUI) có thể sẽ gặp vấn đề khi cùng lúc làm việc với nhiều server. Ta có thể sử dụng T-SQL để làm việc với link server như sau


```sql
-- Bước 1: Tạo link server 

EXEC sp_addlinkedserver     
   @server='RBD',   
   @srvproduct=N'',  
   @provider=N'SQLNCLI',
   @datasrc='10.39.200.204'

-- Bước 2: Tạo password

EXEC sp_addlinkedsrvlogin
  @rmtsrvname ='RBD'
, @useself = 'FALSE'
, @rmtuser = 'bic_user'
, @rmtpassword = 'bic_user'

```

```sql
-- Kiểm tra tất cả link server
sp_helpserver

-- Xóa link server RBD
sp_dropserver 'RBD', 'droplogins';  

```

### Procedure tạo nhiều link server



```sql
drop procedure if exists u_createlinkserver
GO
create procedure u_createlinkserver 
	@name nvarchar(max),
	@source nvarchar(max),
	@user nvarchar(max),
	@password nvarchar(max)
as

BEGIN
	EXEC sp_addlinkedserver     
	   @server=@name,   
	   @srvproduct=N'',  
       @provider=N'SQLNCLI',
	   @datasrc=@source;
   
	EXEC sp_addlinkedsrvlogin
	  @rmtsrvname = @name
	, @useself = 'FALSE'
	, @rmtuser = @user
	, @rmtpassword = @password
END 

```

**Ví dụ**:



```sql
exec u_createlinkserver 
	@name='RBD',   
    @source='10.39.200.204',
	@user = 'bic_user',
	@password = 'bic_user'

```

## Tìm tất cả các bảng, cột trong Database



```sql
SELECT 'SERVER_74' as SERVER,
		'BICDATA' as DATA_BASE,
		lst.name AS Table_Name, 
        lsc.name AS Column_Name, 
        lsc.max_length as Max_Length
FROM [M74].[BICDATA].[sys].[tables] lst
    INNER JOIN [M74].[BICDATA].[sys].[columns] lsc
        ON lst.OBJECT_ID=lsc.object_id
union 
SELECT 'SERVER_16' as SERVER,
	   'VPB_WHR2' as DATA_BASE,
	   lst.name AS Table_Name, 
       lsc.name AS Column_Name, 
       lsc.max_length as Max_Length
FROM [M16].[VPB_WHR2].[sys].[tables] lst
    INNER JOIN [M16].[VPB_WHR2].[sys].[columns] lsc
        ON lst.OBJECT_ID=lsc.object_id
ORDER BY DATA_BASE, lst.name, lsc.name

```

## Chạy file .BAT từ T-SQL



```sql
sp_configure 'show advanced options', '1'
RECONFIGURE
sp_configure 'xp_cmdshell', '1' 
RECONFIGURE

```

```sql
EXEC xp_cmdshell 'cmd /c "F:\D\my_file.bat"'  


```


**Lưu ý**: Cách sử dụng `xp_cmdshell` khắc phục được nhược điểm của `sqlcmd`, cho phép đặt job với file `bat`
